In [2]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 모델 로드
model = BertForSequenceClassification.from_pretrained('./saved_model')
tokenizer = BertTokenizer.from_pretrained('./saved_model')

# 모델 학습
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

# 데이터셋 정의
class HateSpeechDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            return_token_type_ids=False,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        label = self.labels[idx]
        
        # 1~5 label을 True, 0 label을 False로 변환
        if label in [1, 2, 3, 4, 5]:
            label = 1
        else:
            label = 0
        
        return {
            'input_ids': inputs['input_ids'].flatten().to(device),
            'attention_mask': inputs['attention_mask'].flatten().to(device),
            'labels': torch.tensor(label, dtype=torch.long).to(device)
        }
    

# 데이터 로드
df = pd.read_csv('data/output.csv')

# 데이터를 1000개로 제한
X_train, X_test, y_train, y_test = train_test_split(df['text'].sample(1000), df['types'].sample(1000), test_size=0.2, random_state=42)

train_dataset = HateSpeechDataset(X_train.tolist(), y_train.tolist(), tokenizer)
test_dataset = HateSpeechDataset(X_test.tolist(), y_test.tolist(), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


# 평가
model.eval()
predictions, true_labels = [], []
num = 0
for batch in test_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    batch_predictions = torch.argmax(logits, dim=1).tolist()
    batch_true_labels = batch['labels'].tolist()
    
    for i, (text, pred, true_label) in enumerate(zip(X_test, batch_predictions, batch_true_labels)):
        if pred == true_label:
            num += 1
        print(f"Text: {text}")
        print(f"Predicted Label: {pred}")
        print(f"True Label: {true_label}")
        print()

accuracy = num / len(X_test)
print(f"Accuracy: {accuracy:.2f}")


Text: 음식에 양배추 좀 그만 처넣어라
Predicted Label: 0
True Label: 0

Text: 예쁘면 세상 살기 편하더라
Predicted Label: 1
True Label: 0

Text: 뒷광고하는 유튜버들 아직도 활동해?
Predicted Label: 1
True Label: 1

Text: ㄱㄴㄲ; 나중엔 강남 오피녀들도 #@#라고 하겠노
Predicted Label: 0
True Label: 1

Text: 쉴드를 칠 걸 처라
Predicted Label: 1
True Label: 0

Text: 나 나이를 먹을수록 점점 추위를 많이타는 것 같애
Predicted Label: 1
True Label: 1

Text: 그 색희 이제 사회에서도 매장이네
Predicted Label: 0
True Label: 1

Text: 쟤네는 저런 쓰레기 낳아놔서 어떡하냐 ㅋㅋㅋ 불쌍
Predicted Label: 0
True Label: 1

Text: 미쳤다 댓글들 센스 찢었다 진심 ㅋㅋㅋ
Predicted Label: 1
True Label: 1

Text: 부모 얼굴도 모르는 애들이 뭔 힘아리가 있겠냐 ㅋㅋ 고아원에서 주는 밥이나 꾸역꾸역 먹는 애들
Predicted Label: 0
True Label: 0

Text: 진짜 알아서 분리수거함으로 들어가주네요.
Predicted Label: 1
True Label: 0

Text: 쟤 지금 옥수수만 몇개째야?
Predicted Label: 1
True Label: 1

Text: 쟤 분명 나가서 너 예민하다고 욕했을 듯
Predicted Label: 0
True Label: 1

Text: 의료시설도 조선시대라 알아서 저승길 건너서 염라대왕 앞에 직배송될테니까
Predicted Label: 0
True Label: 1

Text: 그래도 계속 벌레인 것보단 나비로 변하는 게 낫지.
Predicted Label: 0
True Label: 1

Text: 니처럼 열심히 해봐야 의미 없다
Pre